In [1]:
!wget https://www.labri.fr/perso/lsimon/option-ia/Search/Master-Class-SAT/pysat.zip
!unzip -o pysat
!mv lorensi-pysat-8625ab1d6cdf/src/* .

--2021-01-31 13:37:17--  https://www.labri.fr/perso/lsimon/option-ia/Search/Master-Class-SAT/pysat.zip
Resolving www.labri.fr (www.labri.fr)... 147.210.8.59, 2001:660:6101:404::80
Connecting to www.labri.fr (www.labri.fr)|147.210.8.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1535196 (1.5M) [application/zip]
Saving to: ‘pysat.zip’

pysat.zip           100%[===================>]   1.46M  2.23MB/s    in 0.7s    

2021-01-31 13:37:18 (2.23 MB/s) - ‘pysat.zip’ saved [1535196/1535196]

Archive:  pysat.zip
  inflating: lorensi-pysat-8625ab1d6cdf/.hg_archival.txt  
  inflating: lorensi-pysat-8625ab1d6cdf/README.md  
  inflating: lorensi-pysat-8625ab1d6cdf/examples/BMC-Unsat/README  
  inflating: lorensi-pysat-8625ab1d6cdf/examples/BMC-Unsat/barrel2.cnf.gz  
  inflating: lorensi-pysat-8625ab1d6cdf/examples/BMC-Unsat/barrel3.cnf.gz  
  inflating: lorensi-pysat-8625ab1d6cdf/examples/BMC-Unsat/barrel4.cnf.gz  
  inflating: lorensi-pysat-8625ab1d6cdf/examples/BMC-Un

In [2]:
raw_package_data = '''
libdatrie-doc:
    DependsOn:
        - libjs-jquery
libjs-jquery:
    Breaks:
        - jquery
        - movabletype-opensource
libfstrcmp0-dbg:
    DependsOn:
        - libfstrcmp0
libfstrcmp0:
    DependsOn:
        - libc6
libc6:
    DependsOn:
        - libgcc1
    Breaks:
        - openrc
        - hurd
        - libtirpc1
        - locales
        - locales-all
        - nscd
libgcc1:
    DependsOn:
        - gcc-8-base
        - openrc
    Breaks:
        - gcc-4.3
        - gcc-4.4
        - gcc-4.5

libtirpc1:
    DependsOn:
        - libc6
        - libgssapi-krb5-2
    Breaks:
        - nfs-common
        - nfs-kernel-server

locales:
    DependsOn:
        - libc-bin
        - libc-bin:i386
        - debconf
        - debconf-2.0
        - cdebconf
        - debconf
    Breaks:
        - libc-bin

locales-all:
    Breaks:
        - locales
nscd:
    DependsOn:
        - lsb-base
        - libaudit1
        - libcap2
        - libselinux1

libgssapi-krb5-2:
    DependsOn:
        - libcom-err2
        - libk5crypto3
        - libkrb5-3
        - libkrb5support0
    Breaks:
        - moonshot-gss-eap

nfs-common:
    DependsOn:
        - libcap2
        - libcom-err2
        - libdevmapper1.02.1
        - libevent-2.1-6
        - libgssapi-krb5-2
        - libkeyutils1
        - libkrb5-3
        - libmount1
        - libnfsidmap2
        - libtirpc1
        - libwrap0
        - rpcbind
        - adduser
        - ucf
        - lsb-base
        - keyutils
        - keyutils:i386
    Breaks:
        - nfs-client
        - nfs-kernel-server
'''

In [3]:
import yaml
package_data = yaml.load(raw_package_data)

In [4]:
all_packages = set()
for package in package_data:
  all_packages.add(package)
  dependencies = package_data[package].get('DependsOn', None)
  conflicts = package_data[package].get('Breaks', None)
  if dependencies is not None:
    all_packages.update(set(dependencies))
  if conflicts is not None:
    all_packages.update(set(conflicts))

all_packages = list(all_packages)
print("[+] There are {} packages in the system".format(all_packages.__len__()))

[+] There are 47 packages in the system


In [7]:
#software_to_install = 'nfs-common'         # cannot install
software_to_install = 'libgssapi-krb5-2' # can install
print("[+] To install {}".format(software_to_install))
print("[..] Checking dependencies..")

CNF = []

from pysat import Solver
solver = Solver()

to_verify = set([software_to_install])
verifed   = []
while to_verify.__len__() > 0:
  software_to_verify = to_verify.pop()
  if software_to_verify in all_packages and software_to_verify not in verifed:
    dependencies = package_data.get(software_to_verify, {}).get('DependsOn', [])
    for dependency in dependencies:
      variable_1 = -all_packages.index(software_to_verify)
      variable_2 = all_packages.index(dependency)
      # print("adding dependencies: {}".format([variable_1, variable_2]))
      CNF.append([variable_1, variable_2])
    CNF.append([all_packages.index(software_to_verify)])
    to_verify.update(dependencies)
    to_verify.__len__()

    conflicts = package_data.get(software_to_verify, {}).get('Breaks', [])
    for conflict in conflicts:
      variable_1 = -all_packages.index(software_to_verify)
      variable_2 = -all_packages.index(conflict)
      # print("adding conflicts: {}".format([variable_1, variable_2]))
      CNF.append([variable_1, variable_2])
    
    verifed.append(software_to_verify)
  elif software_to_verify not in all_packages:
    print("[-] '{}' Not recognized".format(software_to_verify))
  
    

for cnf in CNF:
  solver.addClause(cnf)

print(CNF)
solver.buildDataStructure()
solver.solve()

solution = solver.finalModel
print("[*] Package can be installed? {}".format(solution.__len__() > 0))

[+] To install libgssapi-krb5-2
[..] Checking dependencies..
[[-17, 33], [-17, 4], [-17, 41], [-17, 12], [17], [-17, -26], [12], [33], [4], [41]]
c Building data structures in 0.00s
c Ready to go with 41 variables and 10 clauses
[*] Package can be installed? True
